In [1]:

%load_ext autoreload
%autoreload 2
import random
import numpy as np
import itertools
from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression

In [51]:
indices_used = list(itertools.product(itertools.permutations(range(4), 2), itertools.permutations(range(3), 2), itertools.permutations(range(2), 2)))
operations_used = list(itertools.product(range(4), repeat=3)) # 4 operations, using 3 at a time
operations = [
    lambda x, y: x + y,
    lambda x, y: x - y,
    lambda x, y: x * y,
    lambda x, y: x / y,
]
operation_names = ["+", "-", "*", "/"]
combos = list(itertools.product(indices_used, operations_used))
print("Possible ways to use the 4 numbers, possible operations to use, all combinations")
len(indices_used), len(operations_used), len(indices_used) * len(operations_used)

Possible ways to use the 4 numbers, possible operations to use, all combinations


(144, 64, 9216)

In [59]:
min_val = 1
max_val = 12
target = 24
filename = f"questions{max_val}_{target}.txt"
create_data = False
def get_questions():
    count = 0
    res = []
    for a in tqdm(range(min_val, max_val + 1)):
        count_i = 0
        for b in range(a, max_val + 1):
            for c in range(b, max_val + 1):
                for d in range(c, max_val + 1):
                    count_i += 1
                    count += 1
                    for index_order, operation_order in itertools.product(indices_used, operations_used):
                        numbers = [a, b, c, d]
                        valid = True
                        for (index1, index2), operation_index in zip(index_order, operation_order):
                            val1 = numbers[index1]
                            val2 = numbers[index2]
                            numbers.pop(max(index1, index2))
                            numbers.pop(min(index1, index2))
                            if operation_index == 3 and val2 == 0:
                                valid = False
                                break
                            result = operations[operation_index](val1, val2)
                            if result % 1 != 0 or result < 0:
                                valid = False
                                break
                            numbers.append(result)
                        if valid and numbers[0] % 1 == 0 and numbers[0] == 24:
                            res.append((a, b, c, d, 24))
                            break
        print(count_i)
    print(count)
    return res
if create_data:
    questions = get_questions()
    with open(filename, "w") as file:
        file.write("\n".join([f"{' '.join([str(item) for item in items])}" for items in questions]))
else:
    with open(filename, "r") as file:
        questions = file.read().split("\n")
        questions = [list(map(int, question.split())) for question in questions]
print(len(questions), "questions")
print(questions)

1054 questions
[[1, 1, 1, 8, 24], [1, 1, 1, 11, 24], [1, 1, 1, 12, 24], [1, 1, 2, 6, 24], [1, 1, 2, 7, 24], [1, 1, 2, 8, 24], [1, 1, 2, 9, 24], [1, 1, 2, 10, 24], [1, 1, 2, 11, 24], [1, 1, 2, 12, 24], [1, 1, 3, 4, 24], [1, 1, 3, 5, 24], [1, 1, 3, 6, 24], [1, 1, 3, 7, 24], [1, 1, 3, 8, 24], [1, 1, 3, 9, 24], [1, 1, 3, 10, 24], [1, 1, 3, 11, 24], [1, 1, 3, 12, 24], [1, 1, 4, 4, 24], [1, 1, 4, 5, 24], [1, 1, 4, 6, 24], [1, 1, 4, 7, 24], [1, 1, 4, 8, 24], [1, 1, 4, 9, 24], [1, 1, 4, 10, 24], [1, 1, 4, 12, 24], [1, 1, 5, 5, 24], [1, 1, 5, 6, 24], [1, 1, 5, 7, 24], [1, 1, 5, 8, 24], [1, 1, 6, 6, 24], [1, 1, 6, 8, 24], [1, 1, 6, 9, 24], [1, 1, 6, 12, 24], [1, 1, 7, 10, 24], [1, 1, 8, 8, 24], [1, 1, 10, 12, 24], [1, 1, 11, 11, 24], [1, 1, 11, 12, 24], [1, 1, 12, 12, 24], [1, 2, 2, 4, 24], [1, 2, 2, 5, 24], [1, 2, 2, 6, 24], [1, 2, 2, 7, 24], [1, 2, 2, 8, 24], [1, 2, 2, 9, 24], [1, 2, 2, 10, 24], [1, 2, 2, 11, 24], [1, 2, 2, 12, 24], [1, 2, 3, 3, 24], [1, 2, 3, 4, 24], [1, 2, 3, 5, 24], [1, 2, 

In [60]:
use_random = True
def solve(question):
    # pair up two numbers 1-4, then 1-3, then 1-2.
    # For each pair, we can add, subtract, multiply or divide.
    target = question[-1]
    choices = list(question[:-1]).copy()
    # shuffle it
    if use_random:
        random.shuffle(choices)
    i = 0
    while use_random or i < len(combos):
        index_order, operation_order = random.choice(combos) if use_random else combos[i]
        i += 1
        numbers = choices.copy()
        valid = True
        for (index1, index2), operation_index in zip(index_order, operation_order):
            val1 = numbers[index1]
            val2 = numbers[index2]
            numbers.pop(max(index1, index2))
            numbers.pop(min(index1, index2))
            if operation_index == 3 and val2 == 0:
                valid = False
                break
            numbers.append(operations[operation_index](val1, val2))
        if valid and numbers[0] == target:
            return True, index_order, operation_order, i
    return False, None, None, -1
def print_solution(numbers, index_order, operation_order, answer):
    terms = [str(x) for x in numbers]
    for (index1, index2), operation_index in zip(index_order, operation_order):
        term1 = terms[index1]
        term2 = terms[index2]
        terms.pop(max(index1, index2))
        terms.pop(min(index1, index2))
        new_term = f"({term1} {operation_names[operation_index]} {term2})"
        terms.append(new_term)
    print(" ".join(terms), "=", answer)
debug = False
avg_count = 0
n_entries = 0
solutions = []
min_count = 100000
max_count = 0
for question in tqdm(questions):
    valid, index_order, operation_order, i = solve(question)
    if not valid:
        print("Failed to solve", question)
        break
    else:
        min_count = min(min_count, i)
        max_count = max(max_count, i)
        avg_count += i
        n_entries += 1
        solutions.append([question[:-1], question[-1], index_order, operation_order])
        if debug:
            print("input", question, "indices", index_order, "operations", operation_order, "count", i)
            print_solution(question[0], index_order, operation_order, question[1])
            print()

print("Average count", avg_count / n_entries, "Min", min_count, "Max", max_count) # 1183.991 in order, 661.90353 random

  0%|          | 0/1054 [00:00<?, ?it/s]

Average count 403.58918406072104 Min 1 Max 13962


In [61]:
label_is_3d = True
print(solutions[0])
X = np.zeros((len(solutions), 4))
y = np.zeros((len(solutions), 3, 4)) if label_is_3d else np.zeros((len(solutions), 3*4)) # predict the 3 operations using one hot encoding
for i, sol in enumerate(solutions):
    X[i] = sol[0] # input
    # y[i] = sol[3] # regression output
    # one hot encoding
    for rule_index in range(3):
        if label_is_3d:
            y[i, rule_index, sol[3][rule_index]] = 1
        else:
            y[i, rule_index*4 + sol[3][rule_index]] = 1
print(X.shape, X)
print(y.shape, y)

[[1, 1, 1, 8], 24, ((2, 3), (1, 2), (0, 1)), (0, 0, 2)]
(1054, 4) [[ 1.  1.  1.  8.]
 [ 1.  1.  1. 11.]
 [ 1.  1.  1. 12.]
 ...
 [11. 11. 12. 12.]
 [11. 12. 12. 12.]
 [12. 12. 12. 12.]]
(1054, 3, 4) [[[1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]]

 [[1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]]

 [[0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]]

 ...

 [[0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]]]


In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
class MultiOutputMultiClassClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, base_model=None):
        self.base_model = base_model if base_model is not None else LogisticRegression(max_iter=1000)
    
    def fit(self, X, y):
        n = y.shape[1]
        self.models = [None] * n
        for i in range(n):
            self.models[i] = self.base_model.fit(X, np.argmax(y[:, i, :], axis=1))
        
        return self
    
    def predict(self, X):
        preds = [model.predict(X) for model in self.models] # Predict each output
        return np.stack(preds, axis=1) # stack them together
    
    def predict_proba(self, X):
        probas = [model.predict_proba(X) for model in self.models]
        return np.stack(probas, axis=1)

model = MultiOutputMultiClassClassifier()
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_test)
# print(y_test)
# print(y_pred)

In [ ]:
use_random_weights = True
debug = False
def solve_with_model(x):
    # pair up two numbers 1-4, then 1-3, then 1-2.
    # For each pair, we can add, subtract, multiply or divide.

    target = 24
    choices = list(x)
    # shuffle it
    if use_random_weights:
        random.shuffle(choices)

    i = 0
    y = model.predict_proba([x])[0]
    if debug:
        print(x, y)
    while True:
        index_order = random.choice(indices_used)
        operation_order = [random.choices(range(4), weights=y[op_index], k=1)[0] for op_index in range(3)]
        i += 1
        numbers = choices.copy()
        valid = True
        for (index1, index2), operation_index in zip(index_order, operation_order):
            val1 = numbers[index1]
            val2 = numbers[index2]
            numbers.pop(max(index1, index2))
            numbers.pop(min(index1, index2))
            if operation_index == 3 and val2 == 0:
                valid = False
                break
            numbers.append(operations[operation_index](val1, val2))
        if valid and numbers[0] == target:
            return True, index_order, operation_order, i
    return False, None, None, -1
def print_solution(numbers, index_order, operation_order, answer):
    terms = [str(x) for x in numbers]
    for (index1, index2), operation_index in zip(index_order, operation_order):
        term1 = terms[index1]
        term2 = terms[index2]
        terms.pop(max(index1, index2))
        terms.pop(min(index1, index2))
        new_term = f"({term1} {operation_names[operation_index]} {term2})"
        terms.append(new_term)
    print(" ".join(terms), "=", answer)
for name, X_split, y_split in [("train", X_train, y_train), ("test", X_test, y_test)]:
    # if name == "train":
    #     continue
    print(name)
    avg_count = 0
    n_entries = 0
    for i in tqdm(range(len(X_split))):
        valid, index_order, operation_order, count = solve_with_model(X_split[i])
        if not valid:
            print("Failed to solve", X_split[i])
            break
        else:
            avg_count += count
            n_entries += 1
            if debug:
                print("input", X_split[i], "indices", index_order, "operations", operation_order, "count", i)
                print_solution(X_split[i], index_order, operation_order, 24)
                print()
                if i > 10:
                    break
    print(name, "Average count", avg_count / n_entries) # Train/Test 841.2078 / 766.814

test


  0%|          | 0/211 [00:00<?, ?it/s]

test Average count 777.1421800947867


In [ ]:
label_is_3d = True
print(solutions[0])
X = np.zeros((len(solutions), 5))
# 3 indices, 3 operations
y = np.zeros((len(solutions), 6, 4)) if label_is_3d else np.zeros((len(solutions), 3*4)) # predict the 3 operations using one hot encoding
for i, sol in enumerate(solutions):
    X[i] = sol[0] + [sol[1]] # input + target
    # y[i] = sol[3] # regression output
    # one hot encoding
    for rule_index in range(3):
        if label_is_3d:
            y[i, rule_index, sol[3][rule_index]] = 1
        else:
            y[i, rule_index*4 + sol[3][rule_index]] = 1
print(X.shape, X)
print(y.shape, y)